#Предсказание болезни по симптому на основе метода LR

Импорт библиотек

In [ ]:
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import warnings

from typing import Iterable

Загрузка датасета

In [ ]:
df = pd.read_csv('/content/dataset.csv')
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Проведем предобработку данных

In [ ]:
def string_preparation(st:str) -> str:
    return  st.replace('(',' ')\
            .replace(')',' ')\
            .strip()\
            .replace(' ','_')\
            .replace('__','_')

In [ ]:
def row_to_string(row: Iterable['str'] ) -> str:
    return ' '.join(row.values)

In [ ]:
df_new = (df.drop('Disease', axis = 1).fillna('')).applymap(string_preparation)

In [ ]:
simptoms_list = df_new.apply(row_to_string, axis=1).values.tolist()

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(simptoms_list)

In [ ]:
my_df = pd.DataFrame(X.toarray(), columns= vectorizer.get_feature_names_out())
symptoms_list = vectorizer.get_feature_names_out()

In [ ]:
my_df["Disease"] = df["Disease"].map(str.strip)
my_df.shape

(4920, 132)

Поиск ошибок и дубликатов

In [ ]:
#import enchant
import re

dictionary = enchant.Dict("en_US")
for dis in my_df["Disease"].unique():
    for word in re.split(pattern = r'[\s()]+', string = dis, maxsplit=0, flags=0):
        if word not in ["", "GERD", 'cholestasis', 'spondylosis']and not dictionary.check(word):
            print (word)
            print(dictionary.suggest(word))
            print ("dis*   ", dis)

ImportError: The 'enchant' C library was not found and maybe needs to be installed.
See  https://pyenchant.github.io/pyenchant/install.html
for details


In [ ]:
errors_dict = {
    'Peptic ulcer diseae': 'Peptic ulcer disease',
    'Dimorphic hemmorhoids(piles)': 'Dimorphic hemorrhoids (piles)',
    'Osteoarthristis': 'Osteoarthritis',
    '(vertigo) Paroymsal  Positional Vertigo': '(vertigo) Paroxysmal  Positional Vertigo'
}
def disease_prep(disease):
    if disease in errors_dict:
         return errors_dict[disease]
    return disease
my_df['Disease'] =my_df['Disease'].map(disease_prep)

In [ ]:
dictionary = enchant.Dict("en_US")
for dis in my_df["Disease"].unique():
    for word in re.split(pattern = r'[\s()]+', string = dis, maxsplit=0, flags=0):
        if word not in ["", "GERD", 'cholestasis', 'spondylosis']and not dictionary.check(word):
            print (word)
            print(dictionary.suggest(word))
            print ("dis*   ", dis)

In [ ]:
for sym in my_df.columns:
    for word in sym.split('_'):
        if word not in ["", "diarrhoea", 'polyuria'] and not dictionary.check(word):
            print (word)
            print(dictionary.suggest(word))
            print ("sym*   ", sym)

NameError: name 'dictionary' is not defined

In [ ]:
my_df[my_df['scurring']==1]['Disease'].unique()

array(['Acne'], dtype=object)

In [ ]:
symptoms_errors_dict = {
    'cold_hands_and_feets': 'cold_hands_and_feet',
    'dischromic_patches': 'dyschromic_patches',
    'scurring': 'scarring',
    'swollen_extremeties': 'swollen_extremities',
    'toxic_look_typhos':'toxic_look_typhus'
}
def symptom_prep(symptom):
    if symptom in symptoms_errors_dict:
         return symptoms_errors_dict[symptom]
    return symptom
my_df.columns=my_df.columns.map(symptom_prep)

In [ ]:
for sym in my_df.columns:
    for word in sym.split('_'):
        if word not in ["", "diarrhoea", 'polyuria', 'dyschromic'] and not dictionary.check(word):
            print (word)
            print(dictionary.suggest(word))
            print ("sym*   ", sym)

NameError: name 'dictionary' is not defined

In [ ]:
my_df.to_csv('/content/my_data.my_dataset.csv', index=False)

In [ ]:
my_df[my_df.duplicated ()]

,abdominal_pain,abnormal_menstruation,acidity,acute_liver_failure,altered_sensorium,anxiety,back_pain,belly_pain,blackheads,bladder_discomfort,...,watering_from_eyes,weakness_in_limbs,weakness_of_one_body_side,weight_gain,weight_loss,yellow_crust_ooze,yellow_urine,yellowing_of_eyes,yellowish_skin,Disease
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroxysmal Positional Vertigo
4916,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Acne
4917,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Urinary tract infection
4918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Psoriasis


In [ ]:
df_drop_dup = my_df.drop_duplicates()
print(f' without duplicated shape  {df_drop_dup.shape}')
df_drop_dup.groupby('Disease').Disease.count().sort_values()

 without duplicated shape  (304, 132)


Disease
Gastroenteritis                              5
AIDS                                         5
Acne                                         5
Urinary tract infection                      5
Allergy                                      5
Fungal infection                             5
Heart attack                                 5
Paralysis (brain hemorrhage)                 5
Arthritis                                    6
Cervical spondylosis                         6
Drug Reaction                                6
Impetigo                                     6
Hypertension                                 6
Dimorphic hemorrhoids (piles)                6
(vertigo) Paroxysmal  Positional Vertigo     7
Psoriasis                                    7
Osteoarthritis                               7
Peptic ulcer disease                         7
Hepatitis C                                  7
Bronchial Asthma                             7
GERD                                         7
Malar

In [ ]:
df_drop_dup.to_csv('/content/my_data.my_dataset_small.csv', index=False)

Используем кросс-валидацию

In [ ]:
warnings.filterwarnings("ignore")
X = df_drop_dup.drop("Disease", axis=1)
y = df_drop_dup["Disease"]
n_fold = 7
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state= 11)

In [ ]:
clf  = LogisticRegression(random_state=11)

scores: np.array = cross_val_score(clf, X, y, cv=skf)
print ('all scores ', scores)
print ('mean scores ', round(scores.mean(), 2))

all scores  [1. 1. 1. 1. 1. 1. 1.]
mean scores  1.0


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer

def classification_report_with_accuracy_score(y_true, y_pred):

    print (classification_report(y_true, y_pred)) # print classification report
    return accuracy_score(y_true, y_pred) # return accuracy score

clf = LogisticRegression(random_state=11)

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state= 11)

scores: np.array = cross_val_score(clf, X, y, cv= skf,
                                   scoring=make_scorer(classification_report_with_accuracy_score))

                                          precision    recall  f1-score   support

(vertigo) Paroxysmal  Positional Vertigo       1.00      1.00      1.00         2
                                    AIDS       1.00      1.00      1.00         2
                                    Acne       1.00      1.00      1.00         2
                     Alcoholic hepatitis       1.00      1.00      1.00         2
                                 Allergy       1.00      1.00      1.00         2
                               Arthritis       1.00      1.00      1.00         2
                        Bronchial Asthma       1.00      1.00      1.00         3
                    Cervical spondylosis       1.00      1.00      1.00         2
                             Chicken pox       1.00      1.00      1.00         4
                     Chronic cholestasis       1.00      1.00      1.00         3
                             Common Cold       1.00      1.00      1.00         3
               

In [ ]:
severity = pd.read_csv('/content/Symptom-severity.csv').sort_values('Symptom', ascending=True)
severity["Symptom"] = severity["Symptom"].map(string_preparation)
severity["Symptom"] = severity["Symptom"].map(symptom_prep)
severity

,Symptom,weight
39,abdominal_pain,4
101,abnormal_menstruation,6
8,acidity,3
44,acute_liver_failure,6
98,altered_sensorium,2
...,...,...
19,weight_loss,3
131,yellow_crust_ooze,3
42,yellow_urine,4
43,yellowing_of_eyes,4


In [ ]:
sym_list = df_drop_dup.columns
for sym in sym_list:
    if sym not in severity.Symptom.values:
        print (sym)

foul_smell_of_urine
Disease


In [ ]:
for sym in severity.Symptom.values:
    if sym not in sym_list:
        print (sym)

foul_smell_ofurine
prognosis


In [ ]:
severity.loc[severity['Symptom']=='foul_smell_ofurine', 'Symptom']='foul_smell_of_urine'
for sym in sym_list:
    if sym not in severity.Symptom.values:
        print (sym)

Disease


In [ ]:
severity.to_csv('/content/my_data.my_Symptom_severity.csv', index=False)

In [ ]:
descriptions = pd.read_csv('/content/symptom_Description.csv')
descriptions["Disease"] = descriptions["Disease"].map(str.strip)
descriptions["Disease"] = descriptions["Disease"].map(disease_prep)
descriptions.head()

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


In [ ]:
for dis in df_drop_dup['Disease'].unique():
    if dis not in descriptions['Disease'].values:
        print (dis)

Dimorphic hemorrhoids (piles)


In [ ]:
for dis in descriptions['Disease'].unique():
    if dis not in df_drop_dup['Disease'].values:
        print (dis)

Dimorphic hemorrhoids(piles)


In [ ]:
descriptions.loc[descriptions["Disease"] == 'Dimorphic hemorrhoids(piles)', "Disease"] = 'Dimorphic hemorrhoids (piles)'
for dis in df_drop_dup['Disease'].unique():
    if dis not in descriptions['Disease'].values:
        print (dis)

In [ ]:
new_row = {"Disease":'Not identified. Perhaps you are healthy.',
           'Description': "I can't identify the disease by these symptoms..."\
            " Perhaps you are healthy... or you are still underexamined...)"}
descriptions = pd.concat([descriptions, pd.DataFrame.from_records([new_row])] ,ignore_index=True)
descriptions.tail()

,Disease,Description
37,Pneumonia,Pneumonia is an infection in one or both lungs...
38,Arthritis,Arthritis is the swelling and tenderness of on...
39,Gastroenteritis,Gastroenteritis is an inflammation of the dige...
40,Tuberculosis,Tuberculosis (TB) is an infectious disease usu...
41,Not identified. Perhaps you are healthy.,I can't identify the disease by these symptoms...


In [ ]:
descriptions.to_csv('/content/my_data.my_symptom_description.csv', index=False)

In [ ]:
precautions = pd.read_csv('/content/symptom_precaution.csv')
precautions["Disease"] = precautions["Disease"].map(str.strip)
precautions["Disease"] = precautions["Disease"].map(disease_prep)
for dis in df_drop_dup['Disease'].unique():
    if dis not in precautions['Disease'].values:
        print (dis)

In [ ]:
new_row = {"Disease":'Not identified. Perhaps you are healthy.',
           'Precaution_1': "Enjoy your life!",
           'Precaution_2': "Enjoy your life!!",
           'Precaution_3': "Enjoy your life!!!",
           'Precaution_4': "But do not forget about regular medical examination",}
precautions = pd.concat([precautions, pd.DataFrame.from_records([new_row])] ,ignore_index=True)
precautions.tail()

,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
37,Pneumonia,consult doctor,medication,rest,follow up
38,Arthritis,exercise,use hot and cold therapy,try acupuncture,massage
39,Gastroenteritis,stop eating solid food for while,try taking small sips of water,rest,ease back into eating
40,Tuberculosis,cover mouth,consult doctor,medication,rest
41,Not identified. Perhaps you are healthy.,Enjoy your life!,Enjoy your life!!,Enjoy your life!!!,But do not forget about regular medical examin...


In [ ]:
precautions.to_csv('/content/my_data.my_symptom_precautions.csv', index=False)

In [ ]:
symptom_diagnos = df_drop_dup.groupby(df_drop_dup['Disease']).max()
symptom_diagnos.to_csv('/content/my_data.symptom_diagnos.csv', index=False)
symptom_diagnos

,abdominal_pain,abnormal_menstruation,acidity,acute_liver_failure,altered_sensorium,anxiety,back_pain,belly_pain,blackheads,bladder_discomfort,...,vomiting,watering_from_eyes,weakness_in_limbs,weakness_of_one_body_side,weight_gain,weight_loss,yellow_crust_ooze,yellow_urine,yellowing_of_eyes,yellowish_skin
Disease,,,,,,,,,,,,,,,,,,,,,
(vertigo) Paroxysmal Positional Vertigo,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
AIDS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acne,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Alcoholic hepatitis,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
Allergy,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
Arthritis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bronchial Asthma,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cervical spondylosis,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Chicken pox,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Обучение модели

In [ ]:
clf = LogisticRegression(random_state=11)
clf.fit(my_df.drop("Disease", axis=1), my_df["Disease"])
y=clf.predict_proba(X=my_df.loc[[27]].drop('Disease', axis=1))

df_diagnoses = pd.DataFrame({
    'Diagnosis': clf.classes_,
    'Probability': y[0]
}).sort_values('Probability', ascending=False )
df_diagnoses=df_diagnoses.reset_index(drop=True)
df_diagnoses

,Diagnosis,Probability
0,GERD,0.965260
1,Heart attack,0.008904
2,Drug Reaction,0.002578
3,Gastroenteritis,0.002521
4,Paralysis (brain hemorrhage),0.002266
5,Hypertension,0.001637
6,Peptic ulcer disease,0.001193
7,Bronchial Asthma,0.001027
8,(vertigo) Paroxysmal Positional Vertigo,0.000928
9,Alcoholic hepatitis,0.000911


Сохранение модели для чат-бота

In [ ]:
import pickle
with open('/content/trained_model_LR.pkl','wb') as f:
    pickle.dump(clf, f)